1. linear model is overfitted
2. need to adjust volume data during vacation
3. blend linear prediction and last two hours prediction

1. Create features as hour_min for four hours and three minute slots

1. combi_in_1_t: festival above normal
2. combi_in_2_t: festival below normal
3. combi_in_3_t: festival equals normal
4. combi_out_1_t: festival below normal
5. combi_out_3_t: festival below normal

In [1495]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
pd.set_option('display.max_columns', 500)

import datetime

from __future__ import division

import seaborn as sns

from sklearn import linear_model

In [1496]:
import xgboost as xgb

In [1497]:
def MAPE(preds, outputs):
    preds = np.array(preds)
    outputs = np.array(outputs)
    return np.average(np.abs(outputs - preds) / outputs)

In [1585]:
sma_week_vwt = pd.read_csv('train_vwt.csv')

sma_week_vwt['window_time_formatted'] = sma_week_vwt['window_time_formatted'].apply(
            lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))

sma_week_vwt['date'] = sma_week_vwt['window_time_formatted'].apply(lambda t : t.date())

sma_week_vwt_nfes = sma_week_vwt.copy()
# sma_week_vwt_nfes = sma_week_vwt[sma_week_vwt['is_festival'] == 0]

In [1588]:
combi_in = sma_week_vwt_nfes[sma_week_vwt_nfes['direction'] == 0]
combi_out = sma_week_vwt_nfes[sma_week_vwt_nfes['direction'] == 1]

In [1589]:
combi_in['day_hour'] = (combi_in['month'].astype(int).astype(str) + 
    combi_in['day'].astype(int).astype(str) + combi_in['hour'].astype(int).astype(str)).astype(int)
combi_out['day_hour'] = (combi_out['month'].astype(int).astype(str) +
    combi_out['day'].astype(int).astype(str) + combi_out['hour'].astype(int).astype(str)).astype(int)

C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1590]:
combi_in['month_day'] = (combi_in['month'].astype(int).astype(str) + combi_in['day'].astype(int).astype(str)).astype(int)
combi_out['month_day'] = (combi_out['month'].astype(int).astype(str) +combi_out['day'].astype(int).astype(str)).astype(int)

C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


## Make features

In [1591]:
in_features = ['is_festival', 'is_working_day', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 
              'weekday_4', 'weekday_5', 'weekday_6', 'hour_8', 'hour_9', 'hour_17', 'hour_18',
               'rounded_min_0', 'rounded_min_20', 'rounded_min_40', 
              'wind_direction_0.0', 'wind_direction_15.0', 'wind_direction_30.0', 'wind_direction_45.0', 'wind_direction_60.0', 
              'wind_direction_75.0', 'wind_direction_90.0', 'wind_direction_105.0', 'wind_direction_120.0', 
              'wind_direction_135.0', 'wind_direction_150.0', 'wind_direction_165.0', 'wind_direction_180.0', 
              'wind_direction_195.0', 'wind_direction_210.0', 'wind_direction_225.0', 'wind_direction_240.0', 
              'wind_direction_255.0', 'wind_direction_270.0', 'wind_direction_285.0', 'wind_direction_300.0', 
              'wind_direction_315.0', 'wind_direction_330.0', 'wind_direction_345.0', 'wind_direction_360.0', 
              'wind_speed_0.0', 'wind_speed_1.0', 'wind_speed_2.0', 'wind_speed_3.0', 'wind_speed_4.0', 'wind_speed_5.0', 
              'wind_speed_6.0', 'wind_speed_7.0', 'temperature_14.0', 'temperature_16.0', 'temperature_18.0', 
              'temperature_20.0', 'temperature_22.0', 'temperature_24.0', 'temperature_26.0', 'temperature_28.0', 
              'temperature_30.0', 'temperature_32.0', 'rel_humidity_40.0', 'rel_humidity_50.0', 'rel_humidity_60.0', 
              'rel_humidity_70.0', 'rel_humidity_80.0', 'rel_humidity_90.0', 'precipitation_-0.0', 'precipitation_1.0', 
              'precipitation_2.0', 'precipitation_3.0', 'precipitation_4.0', 'precipitation_5.0', 
              'his_ave', 'traj_count', 'travel_time']

In [1592]:
import copy

out_features = copy.deepcopy(in_features)

out_features.remove('traj_count')
out_features.remove('travel_time')

In [1593]:
for f in in_features:
    if f not in combi_in:
        combi_in[f] = np.array([0] * len(combi_in))
        
for f in out_features:
    if f not in combi_out:
        combi_out[f] = np.array([0] * len(combi_out))

C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Select and split data

In [1594]:
combi_in = combi_in[combi_in['hour'].isin([8, 9, 17, 18])]
combi_out = combi_out[combi_out['hour'].isin([8, 9, 17, 18])]

In [1595]:
combi_in_1 = combi_in[combi_in['tollgate_id'] == 1]
combi_in_2 = combi_in[combi_in['tollgate_id'] == 2]
combi_in_3 = combi_in[combi_in['tollgate_id'] == 3]
combi_out_1 = combi_out[combi_out['tollgate_id'] == 1]
combi_out_3 = combi_out[combi_out['tollgate_id'] == 3]

## Split train and test set

In [1596]:
import random

data_sets = [combi_in_1, combi_in_2, combi_in_3, combi_out_1, combi_out_3]

data_names = ["entry1", 'entry2', 'entry3', 'exit1', 'exit3']

feature_list = []

for i in range(5):
    if i <= 2:
        feature_list.append(in_features)
    else:
        feature_list.append(out_features)

In [1597]:
combi_in['date'].unique()

array([datetime.date(2016, 9, 19), datetime.date(2016, 9, 20),
       datetime.date(2016, 9, 21), datetime.date(2016, 9, 22),
       datetime.date(2016, 9, 23), datetime.date(2016, 9, 24),
       datetime.date(2016, 9, 25), datetime.date(2016, 9, 26),
       datetime.date(2016, 9, 27), datetime.date(2016, 9, 28),
       datetime.date(2016, 9, 29), datetime.date(2016, 9, 30),
       datetime.date(2016, 10, 1), datetime.date(2016, 10, 2),
       datetime.date(2016, 10, 3), datetime.date(2016, 10, 4),
       datetime.date(2016, 10, 5), datetime.date(2016, 10, 6),
       datetime.date(2016, 10, 7), datetime.date(2016, 10, 8),
       datetime.date(2016, 10, 9), datetime.date(2016, 10, 10),
       datetime.date(2016, 10, 11), datetime.date(2016, 10, 12),
       datetime.date(2016, 10, 13), datetime.date(2016, 10, 14),
       datetime.date(2016, 10, 15), datetime.date(2016, 10, 16),
       datetime.date(2016, 10, 17), datetime.date(2016, 10, 18),
       datetime.date(2016, 10, 19), datetime.d

In [1598]:
Xs_train_valid = []
Ys_train_valid = []
Xs_test = []
Ys_test = []
for i in range(5):
    df = data_sets[i]
    features = feature_list[i]
    
    month_day = df['month_day'].unique()
    random.shuffle(month_day)
    test_size = int(0.25 * len(month_day))
    test_index = month_day[:test_size]
    train_index = month_day[test_size:]
    
#     test_index = train_index.copy()
    
    print len(train_index)
    print len(test_index)
    train = df[df['month_day'].isin(train_index)]
    test = df[df['month_day'].isin(test_index)]
    
    Xs_train_valid.append(train[features])
    Ys_train_valid.append(train['count'])
    Xs_test.append(test[features])
    Ys_test.append(test['count'])

27
9
27
9
27
9
27
9
27
9


## Fit linear model

In [1599]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.cross_validation import train_test_split

In [1600]:
from sklearn.tree import DecisionTreeRegressor

def fit_model_DT(X_train_valid, Y_train_valid):
    
    mapes = []
    regs = []
    
    seed = randint(0, 50)
    X_train_valid = X_train_valid.sample(frac=1, random_state=seed)
    Y_train_valid = Y_train_valid.sample(frac=1, random_state=seed)
    
    X_train_valid.index = range(len(X_train_valid))
    Y_train_valid.index = range(len(Y_train_valid))
    
    kf = KFold(n_splits=5)
    
    
    for i in range(10):
        
        depth = 1+i
        mape_list = []
        
        for t_index, v_index in kf.split(X_train_valid):

            train_X = X_train_valid.loc[t_index]
            valid_X = X_train_valid.loc[v_index]
            train_Y = Y_train_valid.loc[t_index]
            valid_Y = Y_train_valid.loc[v_index]

            reg = DecisionTreeRegressor(max_depth=depth)
            reg.fit(train_X, train_Y)
            pred = reg.predict(valid_X)
            mape_list.append(MAPE(pred, valid_Y))
            
        mapes.append(np.average(mape_list))
    
    index = np.array(mapes).argmin()
    best_depth = index + 1
    mape = np.average(mape_list)
    reg = DecisionTreeRegressor(max_depth=best_depth)
    reg.fit(X_train_valid,Y_train_valid)
    
    return (best_depth, mape, reg)               

In [1601]:
from random import randint

def fit_model_LinearReg(X_train_valid, Y_train_valid, minalpha=3, maxalpha=20):
    
    preds = []
    mapes = []
    regs = []
    
    seed = randint(0, 50)
    X_train_valid = X_train_valid.sample(frac=1, random_state=seed)
    Y_train_valid = Y_train_valid.sample(frac=1, random_state=seed)
    
    X_train_valid.index = range(len(X_train_valid))
    Y_train_valid.index = range(len(Y_train_valid))

    alphas = np.arange(minalpha, maxalpha, 0.1)
    
    for a in alphas:
        
        kf = KFold(n_splits=5)
        mape_list = []
        for t_index, v_index in kf.split(X_train_valid):
            
            train_X = X_train_valid.loc[t_index]
            valid_X = X_train_valid.loc[v_index]
            train_Y = Y_train_valid.loc[t_index]
            valid_Y = Y_train_valid.loc[v_index]
            
            reg = linear_model.Ridge(alpha=a)
            reg.fit(train_X, train_Y)
            pred = reg.predict(valid_X)
            mape = MAPE(pred, valid_Y)
            mape_list.append(mape)
        mapes.append(np.average(mape_list))
        
    index = np.array(mapes).argmin()
    alpha = alphas[index]
    mape = mapes[index]
    reg = linear_model.Ridge(alpha=alpha)
    reg.fit(X_train_valid, Y_train_valid)
    
    return (alpha, mape, reg)

In [1602]:
from sklearn.ensemble import GradientBoostingRegressor

In [1603]:
from sklearn.tree import DecisionTreeRegressor

def fit_model_GB(X_train_valid, Y_train_valid):
    
    mapes = []
    regs = []
    
    seed = randint(0, 50)
    
    X_train_valid = X_train_valid.sample(frac=1, random_state=seed)
    Y_train_valid = Y_train_valid.sample(frac=1, random_state=seed)
    
    X_train_valid.index = range(len(X_train_valid))
    Y_train_valid.index = range(len(Y_train_valid))
    
    kf = KFold(n_splits=5)
    
    
    for i in range(3):
        
        depth = 1+i
        mape_list = []
        
        for t_index, v_index in kf.split(X_train_valid):

            train_X = X_train_valid.loc[t_index]
            valid_X = X_train_valid.loc[v_index]
            train_Y = Y_train_valid.loc[t_index]
            valid_Y = Y_train_valid.loc[v_index]

            reg = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=depth, random_state=seed, loss='ls')
            reg.fit(train_X, train_Y)
            pred = reg.predict(valid_X)
            mape_list.append(MAPE(pred, valid_Y))
            
        mapes.append(np.average(mape_list))
    
    index = np.array(mapes).argmin()
    best_depth = index + 1
    mape = np.average(mape_list)
    reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=best_depth, random_state=0, loss='ls')
    reg.fit(X_train_valid,Y_train_valid)
    
    return (best_depth, mape, reg)               

## Train models 

In [1604]:
def train_GB_models(Xs_train_valid, Ys_train_valid, verbose=True):

#     alphas = []
    mapes_validation = []
    GB_models = []
    total_val_mapes = []
    total_test_mapes = []

    for i in range(5):
        (max_depth, mape_valid, reg) = fit_model_GB(Xs_train_valid[i], Ys_train_valid[i])

        alphas.append(alpha)
        mapes_validation.append(mape_valid)
        GB_models.append(reg)

        X_test = Xs_test[i]
        Y_test = Ys_test[i]
        pred = reg.predict(X_test)
        mape_test = MAPE(pred, Y_test)

        total_val_mapes.append(mape_valid)
        total_test_mapes.append(mape_test)

        if verbose:
            print 'max_depth:', max_depth, 'mape_valid:', mape_valid, 'mape_test:', mape_test

    print 'Validation mape:', np.average(total_val_mapes)
    print 'Test mape:', np.average(total_test_mapes)
    
    return GB_models

In [1605]:
def train_LR_models(Xs_train_valid, Ys_train_valid, verbose=True, minalpha=3, maxalpha=20):
    
    alphas = []
    mapes_validation = []
    LR_models = []
    total_val_mapes = []
    total_test_mapes = []
    
    for i in range(5):
        (alpha, mape_valid, reg) = fit_model_LinearReg(Xs_train_valid[i], Ys_train_valid[i], minalpha, maxalpha)

        alphas.append(alpha)
        mapes_validation.append(mape_valid)
        LR_models.append(reg)

        X_test = Xs_test[i]
        Y_test = Ys_test[i]
        pred = reg.predict(X_test)
        mape_test = MAPE(pred, Y_test)

        total_val_mapes.append(mape_valid)
        total_test_mapes.append(mape_test)

        if verbose:
            print 'alpha:', alpha, 'mape_valid:', mape_valid, 'mape_test:', mape_test
            
    print 'Validation mape:', np.average(total_val_mapes)
    print 'Test mape:', np.average(total_test_mapes)
    
    return LR_models

In [1606]:
def train_DT_models(Xs_train_valid, Ys_train_valid, verbose=True):

#     alphas = []
    mapes_validation = []
    DT_models = []
    total_val_mapes = []
    total_test_mapes = []
    
    for i in range(5):
        (max_depth, mape_valid, reg) = fit_model_DT(Xs_train_valid[i], Ys_train_valid[i])

#         alphas.append(alpha)
        mapes_validation.append(mape_valid)
        DT_models.append(reg)

        X_test = Xs_test[i]
        Y_test = Ys_test[i]
        pred = reg.predict(X_test)
        mape_test = MAPE(pred, Y_test)

        total_val_mapes.append(mape_valid)
        total_test_mapes.append(mape_test)

        if verbose:
            print 'max_depth:', max_depth, 'mape_valid:', mape_valid, 'mape_test:', mape_test
            
    print 'Validation mape:', np.average(total_val_mapes)
    print 'Test mape:', np.average(total_test_mapes)
    
    return DT_models

In [1607]:
LR_models = train_LR_models(Xs_train_valid, Ys_train_valid, verbose=True, minalpha=5, maxalpha=20)

alpha: 5.0 mape_valid: 0.248819988118 mape_test: 0.433887594835
alpha: 5.0 mape_valid: 0.52984528638 mape_test: 0.661340645846
alpha: 7.1 mape_valid: 0.145506876466 mape_test: 0.102312775189
alpha: 5.0 mape_valid: 0.312120023306 mape_test: 0.696336983433
alpha: 5.0 mape_valid: 0.393819602436 mape_test: 0.308550481778
Validation mape: 0.326022355341
Test mape: 0.440485696216


In [1608]:
DT_models = train_DT_models(Xs_train_valid, Ys_train_valid, verbose=True)

max_depth: 6 mape_valid: 0.225998129162 mape_test: 0.690080293
max_depth: 8 mape_valid: 0.364040991312 mape_test: 0.553850789107
max_depth: 3 mape_valid: 0.204766972203 mape_test: 0.143392032624
max_depth: 6 mape_valid: 0.195301664148 mape_test: 1.42747225654
max_depth: 8 mape_valid: 0.239391518274 mape_test: 0.175231662803
Validation mape: 0.24589985502
Test mape: 0.598005406814


In [1609]:
GB_models = train_GB_models(Xs_train_valid, Ys_train_valid, verbose=True)

max_depth: 3 mape_valid: 0.215775752698 mape_test: 0.658995961214
max_depth: 3 mape_valid: 0.389057457951 mape_test: 0.497735400268
max_depth: 2 mape_valid: 0.153647680781 mape_test: 0.106890735083
max_depth: 2 mape_valid: 0.194775630021 mape_test: 0.541736258038
max_depth: 3 mape_valid: 0.235721364433 mape_test: 0.189455056674
Validation mape: 0.237795577177
Test mape: 0.398962682255


## Performance

In [562]:
def create_coef_df(reg_model, features):
    coef_df = pd.DataFrame(data={'coef':reg_model.coef_, 'feature':features})
    coef_df.sort('coef', ascending=False, inplace=True)
    return coef_df

In [563]:
def create_performance_df(reg_model, test_set, features, data_name="Default"):
    output = test_set['count']
    pred = reg_model.predict(test_set[features])
    perfor_df = pd.DataFrame(data={'time': test_set['window_time_formatted'], 'output': output, 'pred': pred})
    mape = MAPE(pred, output)
    print "MAPE of" + data_name
    print mape
    return (perfor_df, mape)

In [564]:
def create_performance_matric():
    names = ['Entry 1', 'Entry 2', 'Entry 3', 'Exit 1', 'Exit 3']
    models = [reg_in_1, reg_in_2, reg_in_3, reg_out_1, reg_out_3]
    validation_mapes = [mape_in_1_v, mape_in_2_v, mape_in_3_v, mape_out_1_v, mape_out_3_v]
    test_sets = [combi_in_1_test, combi_in_2_test, combi_in_3_test, combi_out_1_test, combi_out_3_test]
    alphas = [a_in_1, a_in_2, a_in_3, a_out_1, a_out_3]
    test_mapes = []
    for i in range(5):
        model = models[i]
        test_set = test_sets[i]
        if i <=2:
            feature  = in_features
        else: 
            feature = out_features
        pred = model.predict(test_set[feature])
        output = test_set['count']
        test_mapes.append(MAPE(pred, output))
    perf_matric = pd.DataFrame(data={'Data':names, 'alphas':alphas, 'validation_mape':validation_mapes, 'test_mape':test_mapes})
    return perf_matric

## Make submission

### Load data

In [1256]:
sub_vwt = pd.read_csv('sub_vwt.csv')
sub_combi_in = sub_vwt[old_sub_vwt['direction'] == 0]
sub_combi_out = sub_vwt[old_sub_vwt['direction'] == 1]

### Merge and split

In [1257]:
# Fill features
for f in in_features:
    if f not in sub_combi_in.columns:
        sub_combi_in[f] = np.array([0] * len(sub_combi_in))
        
for f in out_features:
    if f not in sub_combi_out.columns:
        sub_combi_out[f] = np.array([0] * len(sub_combi_out))

# Split data by 'tollgate_id'
sub_combi_in_1 = sub_combi_in[sub_combi_in['tollgate_id'] == 1]
sub_combi_in_2 = sub_combi_in[sub_combi_in['tollgate_id'] == 2]
sub_combi_in_3 = sub_combi_in[sub_combi_in['tollgate_id'] == 3]
sub_combi_out_1 = sub_combi_out[sub_combi_out['tollgate_id'] == 1]
sub_combi_out_3 = sub_combi_out[sub_combi_out['tollgate_id'] == 3]

C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Make prediction

In [1258]:
models = GB_models

In [1259]:
sub_combis = [sub_combi_in_1, sub_combi_in_2, sub_combi_in_3, sub_combi_out_1, sub_combi_out_3]

In [1260]:
for i in range(5):
    model = models[i]
    if i <= 2:
        features = in_features
    else:
        features = out_features
    sub_combis[i]['preds'] = model.predict(sub_combis[i][features])

C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Concatenate submission tables

In [1261]:
sub_pred = pd.concat([sub_combi_in_1, sub_combi_in_2, sub_combi_in_3, sub_combi_out_1, sub_combi_out_3])

## Format submission table

In [1262]:
def format_submission(df):        
    
    df['window_time_formatted'] = df['window_time_formatted'].apply(lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
    df['begin'] = df['window_time_formatted'].apply(lambda t: t.strftime('%Y-%m-%d %H:%M:%S'))
    df['end'] = df['window_time_formatted'].apply(lambda t : (t+datetime.timedelta(minutes=20))).apply(
                   lambda t : t.strftime('%Y-%m-%d %H:%M:%S'))

    left = np.array(['['] * len(df))
    comma = np.array([','] * len(df))
    right = np.array([')'] * len(df))

    df['time_window'] = left + df['begin'] + comma + df['end'] + right
    
    return df

In [1263]:
my_submission = format_submission(sub_pred)

### Create submission table

In [1264]:
# Delete not useful data
my_submission = my_submission[['tollgate_id', 'direction', 'time_window', 'preds']]

# Rename column name
my_submission.rename(columns={'preds':'volume'}, inplace=True)

# Convert data type
# my_submission['volume'] = my_submission['volume'].astype(int)

In [1265]:
# Adjust volumn sequence
my_submission = my_submission[['tollgate_id', 'time_window', 'direction', 'volume']]

# Save data
my_submission.to_csv("predictions/GB_prediction_527_with_traj.csv", index=False)

# Load to check
my_submission = pd.read_csv("predictions/GB_prediction_527_with_traj.csv")

In [1267]:
GB_prediction_527 = pd.read_csv("predictions/GB_prediction_527.csv")

In [1272]:
LR_prediction_527 = pd.read_csv("predictions/LR_prediction_527.csv")

In [1269]:
GB_prediction_527.head()

,tollgate_id,time_window,direction,volume
0,1,"[2016-10-25 08:00:00,2016-10-25 08:20:00)",0,50.540210
1,1,"[2016-10-25 08:20:00,2016-10-25 08:40:00)",0,45.203060
2,1,"[2016-10-25 08:40:00,2016-10-25 09:00:00)",0,48.351741
3,1,"[2016-10-25 09:00:00,2016-10-25 09:20:00)",0,53.658167
4,1,"[2016-10-25 09:20:00,2016-10-25 09:40:00)",0,54.168306


In [1266]:
my_submission.head()

,tollgate_id,time_window,direction,volume
0,1,"[2016-10-25 08:00:00,2016-10-25 08:20:00)",0,51.715947
1,1,"[2016-10-25 08:20:00,2016-10-25 08:40:00)",0,46.068362
2,1,"[2016-10-25 08:40:00,2016-10-25 09:00:00)",0,48.214917
3,1,"[2016-10-25 09:00:00,2016-10-25 09:20:00)",0,57.494315
4,1,"[2016-10-25 09:20:00,2016-10-25 09:40:00)",0,57.479330


In [1270]:
MAPE(GB_prediction_527['volume'], my_submission['volume'])

0.020589479641633237

In [1273]:
MAPE(GB_prediction_527['volume'], LR_prediction_527['volume'])

0.0

In [ ]:
subs = []
in_1 = my_submission[(my_submission['tollgate_id'] == 1) & (my_submission['direction'] == 0)]
in_2 = my_submission[(my_submission['tollgate_id'] == 2) & (my_submission['direction'] == 0)]
in_3 = my_submission[(my_submission['tollgate_id'] == 3) & (my_submission['direction'] == 0)]
out_1 = my_submission[(my_submission['tollgate_id'] == 1) & (my_submission['direction'] == 1)]
out_3 = my_submission[(my_submission['tollgate_id'] == 3) & (my_submission['direction'] == 1)]

In [ ]:
sma_week_vwt.head()

In [522]:
vwt = sma_week_vwt.copy()

vwt['window_time_formatted'] = vwt['window_time_formatted'].apply(
            lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))

vwt['day'] = vwt['window_time_formatted'].apply(lambda t : t.day)

In [523]:
vwt = vwt[vwt['is_festival'] == 0]

In [524]:
vwt = vwt[vwt['hour'].isin([8, 9, 17, 18])]

In [525]:
vwt = vwt[['window_time_formatted', 'tollgate_id', 'direction', 'his_ave', 'count']]

In [526]:
sub_vwt = old_sub_vwt.copy()
sub_vwt = pd.merge(sub_vwt[['window_time_formatted', 'tollgate_id', 'direction', 'his_ave']], 
                  new_volume_gp[['window_time_formatted', 'tollgate_id', 'direction', 'count']], 
                  on=['window_time_formatted', 'tollgate_id', 'direction'], suffixes=('', '_y'), how='left')
sub_vwt['window_time_formatted'] = sub_vwt['window_time_formatted'].apply(
            lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
sub_vwt['day'] = sub_vwt['window_time_formatted'].apply(lambda t : t.day)

In [528]:
print MAPE(sub_vwt['his_ave'], sub_vwt['count'])

0.191386063473


In [529]:
print MAPE(vwt['his_ave'], vwt['count'])

0.139660366945


In [ ]:
old_sub_vwt['window_time_formatted'][0]